In [1]:
import tweepy
import pandas as pd
import time

In [2]:
consumer_key = '**************************'
consumer_secret = '**************************'
access_token = '**************************'
access_token_secret = '**************************'

In [3]:
#Authenticate with credentials
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [ ]:
key_word = 'bitcoin'
limit = 1000

def tweet_search(key_word):
    i = 0
    tweets_df = pd.DataFrame(columns = ['Datetime', 'Tweet', 'Username', 'Retweets', 'Followers'])
    for tweet in tweepy.Cursor(api.search, q = key_word, count = 100, lang = 'en', tweet_mode = 'extended').items():
        print('Tweets downloaded:', i, '/', limit, end = '\r')
        if tweet.user.followers_count > 5000 and tweet.retweet_count == 0:
            tweets_df = tweets_df.append({'Datetime': tweet.created_at, 
                                          'Tweet': tweet.full_text, 
                                          'Username': tweet.user.screen_name, 
                                          'Retweets': tweet.retweet_count, 
                                          'Followers': tweet.user.followers_count,}, ignore_index = True)
            i += 1
            if i % 500 == 0:
                time.sleep(30)
        if i >= limit:
            break
        else:
            pass
        
    tweets_df['Datetime'] = pd.to_datetime(tweets_df['Datetime'], format = '%Y.%m.%d %H:%M:%S')    
    tweets_df.set_index('Datetime', inplace = True)
    return tweets_df

tweets_df = tweet_search(key_word)
#tweets_df['Tweet'].tolist()

tweets_df.to_csv(key_word + '.csv', encoding = 'utf-8')